## Introduction:

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Competition Link ==>
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [1]:
## data manipulation
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
## reading the data 
import json
import zipfile
import csv

## plotting .. 
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

## clustering 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

## Feature Selection 
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2



## warnings
import warnings
warnings.filterwarnings("ignore")

ImportError: No module named numpy

In [ ]:
df = pd.read_csv('House_Predict.csv')

In [ ]:
df.head()

In [ ]:
np.shape(df)

In [ ]:
df.columns

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum()

The total number of rows is 1460 .. the number of missing values in(PoolQC) is 1453 .. 

Hence, this coulmn is almost missing ! .. 

We can delete it safely !

In [ ]:
df = df[df.columns.difference(['PoolQC'])]

In [ ]:
np.shape(df)

Column deleted successfully .. 

Let's check the data type of each variable ..

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
## heatmap
corr = df.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)

In [ ]:
corr = corr[corr.columns.difference(['SalePrice'])]
corr

It is obvious that there is no perfect correlation between any of the variables .. This is perfect .. 

This means that each of them adds to the info. related to the dependent variable .. and each of them is important .. 

As mentioned in this tutorial ==> 
https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python

The most sigificant variables are: 
1. GrLivArea
2. TotalBsmtSF
3. OverallQual
4. YearBuilt

let's check their correlation coeffecient values .. 

In [ ]:
print np.corrcoef(df['GrLivArea'], df['TotalBsmtSF']), '\n'
print np.corrcoef(df['OverallQual'], df['TotalBsmtSF']), '\n'
print np.corrcoef(df['YearBuilt'], df['TotalBsmtSF'])

The correlation between the last two variables is higher .. however, none of them is perfectly correlated .. GOOD! 

## Splitting the data:
Let's split the data to train and test .. 

In [ ]:
ntrain = (len(df)*80)/100
df_train, df_test = df[:ntrain], df[ntrain+1:]  ## 80-20 
df_train.head()

In [ ]:
np.shape(df_train)

## Variables Summary .. 

Calculate summary statistics for the numerical variables ..

In [ ]:
df_train.describe()

## Target Variable .. 

SalePrice is the target variable .. let's give it more attention ..

In [ ]:
df_train['SalePrice'].describe()

Well, Minimum price is not equal zero !

In [ ]:
plt.hist(df_train['SalePrice'])
plt.xlabel('Sale Price ($)')
plt.ylabel('Frequency')

The selling price is devaited from the normal .. it's a +ve skewed ..
this means that the majority of the houses are sold by a price higher that $100,000

We have a selling peak at the interval between '$100,000' and '$200,000' (peakedness) 

Let's plot a more appealing histogram ..

In [ ]:
sns.distplot(df_train['SalePrice'])

sns.distplot()

Flexibly plot a univariate distribution of observations.

This function combines the matplotlib hist function (with automatic calculation of a good default bin size) with the seaborn kdeplot() and rugplot() functions. It can also fit scipy.stats distributions and plot the estimated PDF over the data.

Do we have any outliers ?!

In [ ]:
plt.boxplot(df_train['SalePrice'])

The boxplot as well confirms that the data is +ve skewed .. 

## Relationship between SalePrice and sign. vars:
We already highlight 4 most significant vars above .. let's explore the relationship between each of them and the depedent var.

In [ ]:
GrLivArea_SalePrice = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis = 1)
GrLivArea_SalePrice.head()

In [ ]:
GrLivArea_SalePrice.plot.scatter(x = 'GrLivArea', y = 'SalePrice') ## we may add , ylim=(0,800000) as a 3rd parameter .. to limit 

WOW ! .. 
The two variables have +ve linear relationship (Directly correlated) .. 

In [ ]:
TotalBsmtSF_SalePrice = pd.concat([df_train['SalePrice'], df_train['TotalBsmtSF']], axis = 1)
TotalBsmtSF_SalePrice.head()

In [ ]:
TotalBsmtSF_SalePrice.plot.scatter(x = 'TotalBsmtSF', y = 'SalePrice') ## we may add , ylim=(0,800000) as a 3rd parameter .. to limit 

It's also a linear relationship .. however, a more tending to be exponential .. 

At zero there is no relationship between them ..

In [ ]:
OverallQual_SalePrice = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis = 1)
OverallQual_SalePrice.plot.scatter(x = 'OverallQual', y = 'SalePrice') 

In [ ]:
df_train['OverallQual'].head()

This variable contains discrete values .. 

It has no relationship with SalesPrice at each of its data points .. 

This variable represents (Overall material and finish quality) .. let's plot a boxplot at each of its values ..

In [ ]:
OverallQual_SalePrice_ = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis = 1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x = 'OverallQual', y = "SalePrice", data = OverallQual_SalePrice_)

Perfect :) ..

This is the distribution of values at every discrete value of OverallQual, there are outliers standing there .. especially at the value of 9 ..

In [ ]:
YearBuilt_SalePrice = pd.concat([df_train['SalePrice'], df_train['YearBuilt']], axis = 1)
YearBuilt_SalePrice.plot.scatter(x = 'YearBuilt', y = 'SalePrice') 

Also no linear relationship between SalePrice and YearBuilt ! ..

Let's do the same boxplots .. 

In [ ]:
YearBuilt_SalePrice_ = pd.concat([df_train['SalePrice'], df_train['YearBuilt']], axis = 1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x = 'YearBuilt', y = "SalePrice", data = YearBuilt_SalePrice_)
plt.xticks(rotation=90)

Although it's not a strong tendency, someone might say that 'SalePrice' is more prone to spend more money in new stuff than in old relics ..

## Story Telling ..

Stories aside, we can conclude that:

    'GrLivArea' and 'TotalBsmtSF' seem to be linearly related with 'SalePrice'. Both relationships are positive, which means that as one variable increases, the other also increases. In the case of 'TotalBsmtSF', we can see that the slope of the linear relationship is particularly high.
    'OverallQual' and 'YearBuilt' also seem to be related with 'SalePrice'. The relationship seems to be stronger in the case of 'OverallQual', where the box plot shows how sales prices increase with the overall quality.
